```{contents}
```
## Tree of Thought (ToT)

The **Tree of Thought (ToT)** framework is an advanced reasoning paradigm that extends Chain-of-Thought by **exploring multiple reasoning branches simultaneously**, evaluating them, and selecting the most promising path.
LangGraph provides a natural implementation substrate for ToT because it supports **branching, parallel execution, state evaluation, and pruning**.

---

### **1. Motivation: Why Tree of Thought?**

Standard prompting and even ReAct-style loops follow a **single reasoning path**, which is brittle for:

* complex planning
* ambiguous problems
* combinatorial search
* strategic decision making

Tree of Thought instead treats reasoning as a **search problem**.

| Method              | Reasoning Strategy                   |
| ------------------- | ------------------------------------ |
| Chain-of-Thought    | Single linear trace                  |
| ReAct               | Linear with tool feedback            |
| **Tree of Thought** | **Multi-branch search + evaluation** |

---

### **2. Core Components of ToT**

| Component    | Role                            |
| ------------ | ------------------------------- |
| Thought Node | A candidate reasoning step      |
| State        | Holds current partial solution  |
| Expansion    | Generate multiple next thoughts |
| Evaluation   | Score each branch               |
| Selection    | Choose best branches            |
| Pruning      | Drop low-quality branches       |
| Termination  | Detect final solution           |

---

### **3. Mapping ToT onto LangGraph**

| ToT Concept         | LangGraph Construct |
| ------------------- | ------------------- |
| Thought Node        | Graph Node          |
| Reasoning State     | Shared State        |
| Branch Expansion    | Fan-out edges       |
| Parallel Evaluation | Parallel nodes      |
| Branch Selection    | Conditional routing |
| Pruning             | Edge filtering      |
| Search Loop         | Cyclic graph        |

---

### **4. Conceptual Execution Flow**

```
Start
  ↓
Generate Thoughts
  ↓↓↓
Branch A   Branch B   Branch C
  ↓         ↓         ↓
Evaluate A  Evaluate B Evaluate C
   \        |        /
       Select Best K
            ↓
      Expand Again (Loop)
```

---

### **5. State Design**

```python
class State(TypedDict):
    problem: str
    thoughts: list[str]
    scores: dict[str, float]
    best: str
    depth: int
```

---

### **6. Minimal ToT Implementation in LangGraph**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Dict

class State(TypedDict):
    problem: str
    thoughts: List[str]
    scores: Dict[str, float]
    best: str
    depth: int

def generate(state):
    prompt = f"Generate 3 next steps to solve: {state['problem']}"
    thoughts = llm.invoke(prompt).content.split("\n")
    return {"thoughts": thoughts}

def evaluate(state):
    scores = {}
    for t in state["thoughts"]:
        s = llm.invoke(f"Score this thought 0-10: {t}").content
        scores[t] = float(s)
    best = max(scores, key=scores.get)
    return {"scores": scores, "best": best}

def check_depth(state):
    if state["depth"] >= 3:
        return END
    return "generate"

builder = StateGraph(State)
builder.add_node("generate", generate)
builder.add_node("evaluate", evaluate)

builder.set_entry_point("generate")
builder.add_edge("generate", "evaluate")
builder.add_conditional_edges("evaluate", check_depth, {
    "generate": "generate",
    END: END
})

graph = builder.compile()
```

---

### **7. Why LangGraph is Ideal for ToT**

| Capability          | Needed for ToT    |
| ------------------- | ----------------- |
| Cyclic execution    | Multi-step search |
| Conditional routing | Branch selection  |
| Parallel nodes      | Branch expansion  |
| Persistent state    | Search memory     |
| Checkpoints         | Backtracking      |
| Human-in-the-loop   | Branch validation |

---

### **8. Variants of Tree of Thought**

| Variant           | Description                        |
| ----------------- | ---------------------------------- |
| Breadth-First ToT | Explore all branches at each depth |
| Depth-First ToT   | Explore one branch deeply          |
| Beam Search ToT   | Keep top-K branches                |
| Monte Carlo ToT   | Randomized sampling                |
| Heuristic ToT     | Domain-specific scoring            |
| Hybrid ToT        | Combine strategies                 |

---

### **9. Practical Applications**

* Mathematical proof generation
* Strategic planning
* Code synthesis
* Game playing
* Scientific reasoning
* Business decision support

---

### **10. Mental Model**

Tree of Thought transforms LLMs from **text generators** into **search-based problem solvers**:

> **Reasoning = Search over possible thoughts**

LangGraph provides the execution substrate that makes this search **explicit, controllable, safe, and scalable**.


### Demonstration

In [2]:
# ===================== Tree of Thought with LangGraph =====================

from typing import TypedDict, List, Dict
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# --------------------- State Definition ---------------------

class State(TypedDict):
    problem: str
    thoughts: List[str]
    scores: Dict[str, float]
    best: str
    depth: int

# --------------------- Nodes ---------------------

def generate(state: State):
    prompt = f"""
    Generate 3 possible next reasoning steps for solving:
    {state['problem']}
    """
    output = llm.invoke(prompt).content
    thoughts = [t.strip() for t in output.split("\n") if t.strip()]
    return {"thoughts": thoughts}

def evaluate(state: State):
    scores = {}
    for t in state["thoughts"]:
        s = llm.invoke(f"Score this step from 0-10: return only integer\n{t}").content
        scores[t] = float(s)
    best = max(scores, key=scores.get)
    return {"scores": scores, "best": best, "depth": state["depth"] + 1}

def route(state: State):
    if state["depth"] >= 3:
        return END
    return "generate"

# --------------------- Graph ---------------------

builder = StateGraph(State)
builder.add_node("generate", generate)
builder.add_node("evaluate", evaluate)

builder.set_entry_point("generate")
builder.add_edge("generate", "evaluate")
builder.add_conditional_edges("evaluate", route, {"generate": "generate", END: END})

graph = builder.compile()

# --------------------- Run ---------------------

result = graph.invoke({
    "problem": "Design a plan to reduce traffic congestion in a large city",
    "thoughts": [],
    "scores": {},
    "best": "",
    "depth": 0
})

print("\nFINAL BEST IDEA:\n", result["best"])



FINAL BEST IDEA:
 1. **Conduct a Traffic Analysis**: Gather data on current traffic patterns, peak congestion times, and the most congested areas in the city. Utilize tools like traffic modeling software and surveys to identify the causes of congestion, such as road capacity, public transportation availability, and urban layout.
